# Read atmospheric profiles of Ermida

In [ ]:
import numpy as np
from netCDF4 import Dataset
import pandas as pd
import glob
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt

import sys
rttov_installdir = '/Users/huanyu/Desktop/rttov/' # replace with your own RTTOV installation directory
sys.path.append(rttov_installdir + '/wrapper')
import pyrttov

In [2]:
def read_profile_ermida(file_path, return_info=False):
    data_dict = {}
    # read netCDF file
    nc = Dataset(file_path)
    # read time and location
    data_dict['time'] = nc.variables['time'][:]
    data_dict['lat'] = nc.variables['latitude'][:]
    data_dict['lon'] = nc.variables['longitude'][:]
    # read surface and vertically integrated variables
    data_dict['sp'] = nc.variables['sp'][:]/100 # Pa -> hPa
    data_dict['skt'] = nc.variables['skt'][:]
    data_dict['t2m'] = nc.variables['t2m'][:]
    data_dict['lcc'] = nc.variables['lcc'][:]
    data_dict['tcwv'] = nc.variables['tcwv'][:] # kg/m2 == mm
    # read atmospheric profile
    data_dict['p'] = nc.variables['p'][:]/100 # Pa -> hPa
    data_dict['t'] = nc.variables['t'][:]
    data_dict['q'] = nc.variables['q'][:]
    data_dict['o3'] = nc.variables['o3'][:]

    info = nc.variables.copy()
    # close file
    del nc
    if return_info:
        return data_dict, info
    else:
        return data_dict

In [ ]:
files_profile = glob.glob('data/era5_profile/*.nc')
files_profile.sort()

data_dict_all = {}
for file in files_profile:
    if file == files_profile[0]:
         data_dict, info = read_profile_ermida(file, return_info=True)
    else:
        data_dict = read_profile_ermida(file)
    if file == files_profile[0]:
        for key in data_dict.keys():
            data_dict_all[key] = []
    for key in data_dict.keys():
        data_dict_all[key].append(data_dict[key])

time = np.concatenate(data_dict_all['time']).data # hours since 1900-01-01
lat = np.concatenate(data_dict_all['lat']).data
lon = np.concatenate(data_dict_all['lon']).data
sp = np.concatenate(data_dict_all['sp']).data
skt = np.concatenate(data_dict_all['skt']).data
t2m = np.concatenate(data_dict_all['t2m']).data
lcc = np.concatenate(data_dict_all['lcc']).data
tcwv = np.concatenate(data_dict_all['tcwv']).data
p = np.concatenate(data_dict_all['p'], axis=1).data.T
t = np.concatenate(data_dict_all['t'], axis=1).data.T
q = np.concatenate(data_dict_all['q'], axis=1).data.T # specific humidity
o3 = np.concatenate(data_dict_all['o3'], axis=1).data.T # mixing ratio

q[q <= 0] = 0.1000E-10
o3[o3 <= 0] = 0.1000E-10
num_profiles = len(time)

'''
# convert unit of o3 from mixing ratio to specific humidity
# mixing ratio = mass of substance / mass of dry air
# specific humidity = mass of substance / mass of moist air
w_h2o = (1-q)/q
q_o3 = 1 / (1/o3 + w_h2o/o3)
'''

# convert time
# The full date will be used to calculate the TOA solar irradiance for solar-affected simulations. 
# The time is not currently used by RTTOV so can be zero
dt = pd.to_timedelta(time, unit='h')
dates = pd.Timestamp('1900-01-01') + dt
years = dates.year.to_numpy()
months = dates.month.to_numpy()
days = dates.day.to_numpy()
hours = dates.hour.to_numpy()

# convert specific humidity to relative humidity (RH), and then classify clear/cloudy profiles
RH = 0.263 * p*100 *q /np.exp((17.67*(t-273.16)) / (t-29.65))
idx_clear = np.all(RH<=90, axis=1)

# read geopotential height, and convert to elevation
nc_obj = Dataset('data/profile_info/geopotential.nc')
geopotential = nc_obj.variables['z'][:] # unit: m^2/s^2
# lat, lon = nc_obj.variables['latitude'][:], nc_obj.variables['longitude'][:]
nc_obj.close()
geopotential = geopotential/9.80665/1e3 # unit: km
radius = 6371.229
elevation = radius * geopotential / (radius - geopotential)
elevation = elevation.data[0]

# assign elevation for each profile
row = ((90-lat)/0.25).astype(int)
col = (lon/0.25).astype(int)
h = elevation[row, col]

In [ ]:
# save the basic information for each profile
np.save('data/profile_info/idx_clear.npy', idx_clear)
np.save('data/profile_info/tpw.npy', tcwv.astype(np.float32))
np.save('data/profile_info/skt.npy', skt.astype(np.float32)) # N * 6
np.save('data/profile_info/lcc.npy', lcc.astype(int))
np.save('data/profile_info/lat.npy', lat.astype(np.float32))
np.save('data/profile_info/lon.npy', lon.astype(np.float32))
# np.save('data/lat_month.npy', np.array([lat[idx_clear], months[idx_clear]]).T)

# run rttov

In [ ]:
# Define function to generate a [nprof, nlevels] array based on a [nlevels] array
def expand2nprofiles(n, nprof):
    outp = np.empty((nprof, len(n)), dtype=n.dtype)
    for i in range(nprof):
        outp[i, :] = n[:]
    return outp

def run_rttov(nchans, chan_list, angle, p, t, q, sp, t2m, skt, lat, lon, h, year, month, day, hour):
    # run rttov and get three atmospheirc parameters
    
    nprofiles, nlevels = t.shape
    #Set up the profiles
    myProfiles = pyrttov.Profiles(nprofiles, nlevels)
    myProfiles.GasUnits = 1
    myProfiles.P = p # expand2nprofiles(p, nprofiles)
    myProfiles.T = t
    myProfiles.Q = q
    myProfiles.Angles = angle # expand2nprofiles(angle, nprofiles)

    s2m = np.zeros((nprofiles,6))
    s2m[:,0] = sp
    s2m[:,1] = t2m
    #q2m is not required by setting options use_q2m to false
    myProfiles.S2m = s2m

    skin = np.zeros((nprofiles,9))
    skin[:,0] = skt
    myProfiles.Skin = skin

    surftype = np.array([0,0],dtype=np.int32)
    myProfiles.SurfType = expand2nprofiles(surftype, nprofiles)

    surfgeom = np.zeros((nprofiles,3))
    surfgeom[:,0] = lat
    surfgeom[:,1] = lon
    surfgeom[:,2] = h # indeed RTTOV does not use this value at present, it determines the height from surface pressure
    myProfiles.SurfGeom = surfgeom

    datetimes = np.zeros((nprofiles,6))
    datetimes[:,0] = year
    datetimes[:,1] = month
    datetimes[:,2] = day
    datetimes[:,3] = hour
    myProfiles.DateTimes = datetimes

    #Set up RTTOV instance
    Rttov = pyrttov.Rttov()
    Rttov.FileCoef = '{}/{}'.format(rttov_installdir,"rtcoef_rttov13/rtcoef_visir_rttov13pred54L_o3co2/rtcoef_iss_1_ecostres_o3co2.dat")
    Rttov.Options.AddInterp = True
    Rttov.Options.InterpMode = 4
    Rttov.Options.StoreTrans = True
    Rttov.Options.StoreRad = True
    Rttov.Options.StoreRad2 = True
    Rttov.Options.VerboseWrapper = True
    Rttov.Options.UseQ2m = False
    Rttov.Options.ApplyRegLimits = True
    Rttov.Options.Nthreads = 4
    Rttov.Options.NprofsPerCall = 15

    try:
        Rttov.loadInst(chan_list)
    except pyrttov.RttovError as e:
        sys.stderr.write("Error loading instrument(s): {!s}".format(e))
        sys.exit(1)

    Rttov.Profiles = myProfiles

    # Load emissivity atlases or supply emissivity by RTTOV
    surfemisrefl = np.zeros((5,nprofiles,nchans), dtype=np.float64)
    Rttov.SurfEmisRefl = surfemisrefl # Set surface emissivity to 0 in order to omit surface emittances
    # surfemisrefl[:,:,:] = -1.

    #Run RTTOV
    try:
        Rttov.runDirect()
    except pyrttov.RttovError as e:
        sys.stderr.write("Error running RTTOV direct model: {!s}".format(e))

    trans = Rttov.TauTotal
    upclear = Rttov.Rad2UpClear
    dnclear = Rttov.Rad2DnClear
    
    return trans, upclear, dnclear

In [ ]:
# ECOSTRESS bands
nchans, chan_list = 5, (1,2,3,4,5)
lamda_c = np.array([8.29587257, 8.81448846, 9.21205838, 10.49186976, 12.08465478])
wn_c = 1e4 / lamda_c

nprofiles, nlevels = p.shape

np.random.seed(42)
# define angles for each profile
# vza_min, vza_max = 25, 30
# vza = np.random.uniform(vza_min, vza_max, (nprofiles,1))
vaas = np.zeros((nprofiles,1))
szas = np.ones((nprofiles,1)) * 45
saas = np.ones((nprofiles,1)) * 180

angles_all = []

# for random vza
vzas_all = np.load('data/profile_info/vzas.npy')
num_vzas = vzas_all.shape[1]
for i in range(num_vzas):
    vzas = vzas_all[:,i].reshape(-1,1)
    angles = np.concatenate((vzas, vaas, szas, saas), axis=1)
    angles_all.append(angles)
    
'''
# for fixed vza
for i in range(0, 35, 5):
    vzas = np.ones((nprofiles,1)) * i
    angles = np.concatenate((vzas, vaas, szas, saas), axis=1)
    angles_all.append(angles)
'''

# for Ld calculation, fix vza to 53 deg
vzas = np.ones((nprofiles,1)) * 53
angles_all.append(np.concatenate((vzas, vaas, szas, saas), axis=1))

# add errors (fixed ratio error)
error_q = 0.2 # from -20% to +20% with a step of 5%
q_noise = q * (1+error_q)
# error_t = 2
# t_noise = t + error_t

# add random errors
# error_q = np.random.uniform(0.8, 1.2, size=len(q))
# q_noise = q * error_q[:, None]

In [19]:
# run RTTOV
tau_all, Lu_all = [], []
for i in tqdm(range(len(angles_all))):
    tau, Lu, Ld = run_rttov(nchans, chan_list, angles_all[i], p, t, q_noise, sp, t2m, skt[:,3], lat, lon, h, years, months, days, hours)
    
    if i<len(angles_all)-1:
        # convert the unit of radiances from mW/m2/sr/cm-1 to W/m2/sr/um
        Lu = Lu / 1e3 * wn_c**2 / 1e4
        tau_all.append(tau)
        Lu_all.append(Lu)
    
    if i==len(angles_all)-1:
        Ld = Ld / 1e3 * wn_c**2 / 1e4
        

  0%|          | 0/7 [00:00<?, ?it/s]Load successful >>>>> inst_id : 1, nchannels : 5.
 2024/11/05  15:16:20  Load coefficients:
 2024/11/05  15:16:20  /Users/huanyu/Desktop/rttov//rtcoef_rttov13/rtcoef_visir_rttov13pred54L_o3co2/rtcoef_iss_1_ecostres_o3co2.dat
 2024/11/05  15:16:21  Running RTTOV using nthreads =    4 and nprofs_per_call =       15
Deallocating this inst_id.
 2024/11/05  15:16:34  Dropping coefficients:
 2024/11/05  15:16:34  /Users/huanyu/Desktop/rttov//rtcoef_rttov13/rtcoef_visir_rttov13pred54L_o3co2/rtcoef_iss_1_ecostres_o3co2.dat
 14%|█▍        | 1/7 [00:14<01:25, 14.32s/it]Load successful >>>>> inst_id : 1, nchannels : 5.
 2024/11/05  15:16:35  Load coefficients:
 2024/11/05  15:16:35  /Users/huanyu/Desktop/rttov//rtcoef_rttov13/rtcoef_visir_rttov13pred54L_o3co2/rtcoef_iss_1_ecostres_o3co2.dat
 2024/11/05  15:16:36  Running RTTOV using nthreads =    4 and nprofs_per_call =       15
Deallocating this inst_id.
 2024/11/05  15:16:46  Dropping coefficients:
 2024/11/

In [ ]:
atm_params = {}
tau_array, Lu_array = np.array(tau_all), np.array(Lu_all)
tau_array, Lu_array = np.swapaxes(tau_array, 0, 1), np.swapaxes(Lu_array, 0, 1)
tau_array, Lu_array, Ld = tau_array[idx_clear], Lu_array[idx_clear], Ld[idx_clear]
atm_params['tau'] = tau_array.astype(np.float32)
atm_params['Lu'] = Lu_array.astype(np.float32)
atm_params['Ld'] = Ld.astype(np.float32)

pickle.dump(atm_params, open('data/atm_params/atm_params_p20.pkl', 'wb'))